In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import yfinance as yf
%matplotlib inline

In [ ]:
start = datetime.datetime(1982,12, 31)
end = datetime.datetime(2023, 12, 31)

In [ ]:
SP500 = yf.download('^GSPC', start, end)

In [ ]:
SP500.to_csv('SP500.csv')

In [ ]:
#SP500 = pd.read_csv('SP500.csv')

In [ ]:
SP500.head()

In [ ]:
SP500['Adj Close'].head()

In [ ]:
SP500['Adj Close'].tail()

Calcoliamo i rendimenti semplici lordi e netti e logaritmici

In [ ]:
SP500['RS_Lordo'] = SP500['Adj Close']/SP500['Adj Close'].shift(1)

In [ ]:
SP500.head()

In [ ]:
SP500['RS_Netto'] = SP500['Adj Close'].pct_change(1)

In [ ]:
SP500.head()

In [ ]:
SP500['RL'] = np.log(SP500['Adj Close']/SP500['Adj Close'].shift(1))

In [ ]:
SP500.head()

Qual é il collegamento fra rendimenti semplici e logaritmici.
Aumentando la frequenza di capitalizzazione il rendimento composto semplice si avvicina a quello logaritmico.


In [ ]:
f = [1, 2, 4, 12, 52, 365, 730, 1460]
r = 0.10
tassi = []

for i in f:
    tasso = (1+r/i)**i - 1
    tassi.append(tasso)

for i in tassi:
    print(round(i, 7))

print(round(np.exp(r)-1, 7))

Differenze fra rendimenti semplici e logaritmici

o	Rendimenti semplici non si sommano nel tempo, logaritmici sì

o	Rendimenti semplici si sommano nel portafoglio, logaritmici no

o	Rendimenti semplici sono sensibili alla frequenza di calcolo, logaritmici no


Il prodotto dei rendimenti lordi è il valore capitalizzato dell'investimento di una unità al rendimento della serie

In [ ]:
SP500['Cum'] = np.cumprod(SP500['RS_Lordo'])

In [ ]:
SP500.head()

In [ ]:
SP500.tail()

In [ ]:
plt.plot(SP500['Cum'])
plt.grid()

In [ ]:
SP500['Adj Close'][-1]

In [ ]:
SP500['Adj Close'][0]

In [ ]:
SP500['Adj Close'][0]*SP500['Cum'][-1]

In [ ]:
SP500['Adj Close'][-1]/SP500['Adj Close'][0]

In [ ]:
SP500['Cum'][-1]

Calcoliamo il rendimento composto annuo dell'investimento sul periodo considerato

In [ ]:
from datetime import timedelta

In [ ]:
delta = end - start
delta.days

In [ ]:
t = delta/datetime.timedelta(days=1)

In [ ]:
t

In [ ]:
type(t)

Giorni esatti

In [ ]:
t/365

In [ ]:
SP500_cagr = (SP500['Adj Close'][-1]/SP500['Adj Close'][0])**(365/t) -1
SP500_cagr

Approssimazione 41 anni

In [ ]:
SP500['Cum'][-1]**(1/41) - 1

Per avere una serie più maneggevole passiamo da una frequenza giornaliera a una frequenza mensile.
Utilizzamo il metodo "groupby" e la frequenza mensile ('M'). Poichè vogliamo il valore di fine mese specifichiamo ".last()".
Attenzione: nelle analisi di serie storiche utilizzare sempre valori osservati (giorni, fine settimana, fine mese eccetera) mai medie (".mean()") perché creiamo valori che non corrispondono a veri investimenti e riduciamo artificiosamente la volatilità

In [ ]:
SP500_m = SP500.groupby(pd.Grouper(freq='M')).last() #aggregazione mensile
SP500_m.head()

In [ ]:
SP500_m['RS_Lordo']=SP500_m['Adj Close']/SP500_m['Adj Close'].shift(1)

In [ ]:
SP500_m['RS_Netto'] = SP500_m['Adj Close'].pct_change(1)

In [ ]:
SP500_m['RL']= np.log(SP500_m['Adj Close']/SP500_m['Adj Close'].shift(1))

In [ ]:
SP500_m.head()

In [ ]:
SP500_m['Adj Close'][-1]/SP500_m['Adj Close'][0]

Prima differenza: Rendimenti semplici non si sommano nel tempo, logaritmici sì
Se prendiamo il rendimento semplice medio e lo capitalizziamo per il numero di periodi (41 anni per 12 mesi = 492) otteniamo un valore finale miolto più elevato di quello corretto.
Se facciamo la stessa cosa con il rendimento logaritmico medio otteniamo il valore corretto.

In [ ]:
RS_Netto_m = np.mean(SP500_m['RS_Netto'])
RS_Netto_m

In [ ]:
RL_m = np.mean(SP500_m['RL'])
RL_m

In [ ]:
(1+RS_Netto_m)**(41*12)

In [ ]:
np.exp(RL_m*41*12)

Possiamo passare da rendimenti semplici a rendimenti logaritmici grazie a un risultato ricavato dal cosiddetto Lemma di Ito per il quale RL = RS - 0.5*Var(RS)

In [ ]:
RS_Netto_var = np.var(SP500_m['RS_Netto'])

In [ ]:
RS_Netto_m - 0.5*RS_Netto_var

Rendimenti e portafogli

La media ponderata dei rendimenti semplici delle componenti di un portafoglio è uguale al rendimento del portafoglio

In [ ]:
weights = np.array([0.2,0.5,0.3])

In [ ]:
inizio = np.array([100, 100, 100])

In [ ]:
port_inizio = np.sum(weights*inizio)
port_inizio

In [ ]:
fine = np.array([102,105,115])

In [ ]:
ret_s = fine/inizio -1

In [ ]:
ret_s

In [ ]:
port_fine_s = np.sum(weights*fine)
port_fine_s

In [ ]:
ret_port = np.sum(weights*ret_s)
ret_port

In [ ]:
port_fine_s_check = port_inizio * (1+ret_port)
port_fine_s_check

In [ ]:
port_fine_s == port_fine_s_check

La media ponderata dei rendimenti logaritmici delle componenti di un portafoglio è diversa dal rendimento logaritmico del portafoglio

In [ ]:
ret_l = np.log(fine/inizio)
ret_l

In [ ]:
ret_pond_log = np.sum(ret_l*weights)
ret_pond_log

In [ ]:
ret_port_log = np.log(107.4/100)
ret_port_log

In [ ]:
ret_port_log == ret_pond_log

#### Valori finali di un percorso di accumulazione e distribuzione log normale


In [ ]:
ret = RS_Netto_m
ret

In [ ]:
std = np.sqrt(RS_Netto_var)
std

Creaimo 10000 percorsi di investimento mensili per 10 anni utilizzando media e deviazione standard storiche del S&P500 con un metodo chiamato simulazione MonteCarlo

In [ ]:
n = 10000
T = 120
Port = np.zeros((T+1, n))
Port[0] = 100
for t in range(1, T+1):
    Port[t] = Port[t-1]*(1+ (ret + std*np.random.randn(n)))

plt.figure(figsize = (10,6))
plt.hist(Port[-1], bins = 100)
plt.xlabel('Valore finale')
plt.ylabel('Frequenza')
plt.grid()


In [ ]:
port_log = np.log(Port[-1])

In [ ]:
plt.figure(figsize = (10,6))
plt.hist(port_log, bins = 100)
plt.xlabel('Valore finale')
plt.ylabel('Frequenza')
plt.grid()
